In [1]:
from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

from llama_index.core import Settings
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

Settings.llm = AzureOpenAI(
    model="gpt-4-32k",
    deployment_name="",
    api_key="",
    azure_endpoint="",
    api_version="2024-02-15-preview",
)
Settings.embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="",
    api_key="",
    azure_endpoint="",
    api_version="2024-02-15-preview",
)

In [3]:
def TalktoWeb(website:str,query:str)->str:
    
    """
    website(str) : Link of the webpage
    query(str) : Question about the webpage
    
    Use this function to talk with a webpage
    """
    
    documents = SimpleWebPageReader(html_to_text=True).load_data(
        [website]
    )
    index = SummaryIndex.from_documents(documents)
    # set Logging to DEBUG for more detailed outputs
    query_engine = index.as_query_engine()
    response = query_engine.query(query)
    return str(response)

In [9]:
import csv
def ConvertToCsv(data:str,output_path:str)->str:
    
    """
    data(str) : This contains the csv in string format with each rows seperated by a \n
    output_path(str) : The path where the csv should be stored (Eg: doc.csv)
    
    Use this function to convert string to csv    
    """
    lines = data.split('\n')

    with open(output_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for line in lines:
            fields = line.split(',')
            csv_writer.writerow(fields)

    print("CSV file has been created successfully.")
    

In [4]:
from llama_index.core.tools import FunctionTool

In [5]:
webTool = FunctionTool.from_defaults(fn=TalktoWeb)

In [10]:
csvTool = FunctionTool.from_defaults(fn=ConvertToCsv)

In [11]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [webTool,csvTool], 
    verbose=True
)
agent = AgentRunner(agent_worker,verbose=True)

In [13]:
response = agent.query("""
I need to know some information about https://www.gnani.ai/
You have three tasks

Task - 1:
You should go through the website and find the link that leads to the employees page or the leaders page. 
use the webtool you have. ask the web tool to give you the link of the particular page otherwise it might just 
give you the title of the page. ask where the anchor tags lead to

Task - 2:
You should go through the information of the company members using the link and 
return me a csv in the format(Email,First Name,Last Name,Company,Description)
To craft an email of a person use this method (firstname.lastname@companywebsite)

Task - 3:
write the csv to a file locally and its name should be <company-name>.csv
""")

> Running step 3ce50a5c-d846-47c6-ba07-bd74b6d3df5e. Step input: 
I need to know some information about https://www.gnani.ai/
You have three tasks

Task - 1:
You should go through the website and find the link that leads to the employees page or the leaders page. 
use the webtool you have. ask the web tool to give you the link of the particular page otherwise it might just 
give you the title of the page. ask where the anchor tags lead to

Task - 2:
You should go through the information of the company members using the link and 
return me a csv in the format(Email,First Name,Last Name,Company,Description)
To craft an email of a person use this method (firstname.lastname@companywebsite)

Task - 3:
write the csv to a file locally and its name should be <company-name>.csv

Added user message to memory: 
I need to know some information about https://www.gnani.ai/
You have three tasks

Task - 1:
You should go through the website and find the link that leads to the employees page or the lead

In [14]:
import pandas as pd

In [17]:
df = pd.read_csv("gnani.csv",header=None)

In [18]:
df

,0,1,2,3,4
0,ganesh.gopalan@gnani.ai,Ganesh,Gopalan,Gnani.ai,Co-Founder and CEO with over 25 years of Techn...
1,ananth.nagaraj@gnani.ai,Ananth,Nagaraj,Gnani.ai,Co-Founder and CTO with over 12 years of exper...
2,bharath.shankar@gnani.ai,Bharath,Shankar,Gnani.ai,VP of Engineering leading the designing and ar...
3,sanjay.pai@gnani.ai,Sanjay,Pai,Gnani.ai,CRO and a Sales & Business development leader ...
4,lakshmi.narayanan@gnani.ai,Lakshmi,Narayanan,Gnani.ai,Board advisor and former Chief Executive Offic...
5,mahesh.ramachandran@gnani.ai,Mahesh,Ramachandran,Gnani.ai,Board advisor and a successful Serial and Soci...
